In [72]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [73]:
peo_usage = pd.read_csv("data/user_profile/PEO_TV_Usage_CF.csv")

In [74]:
user_all_df = pd.read_csv("data/user_profile/account_no_map_all.csv")

In [75]:
user_profile = pd.read_csv("data/user_profile/prepared/User_Profile_Null_Handled.csv")

In [76]:
peo_usage.drop(["Unnamed: 0"],axis=1,inplace=True)
user_all_df.drop(["Unnamed: 0"],axis=1,inplace=True)
user_profile.drop(["Unnamed: 0"],axis=1,inplace=True)

In [77]:
peo_usage = peo_usage.merge(user_all_df[["event_source.hash","ACCOUNT_NUM.hash"]], on="event_source.hash", how="left")
peo_usage.drop(["event_source.hash"],axis=1, inplace=True)
peo_usage.drop_duplicates(["ACCOUNT_NUM.hash","usage","rating","year-month"],inplace=True)
peo_usage.dropna(subset=["ACCOUNT_NUM.hash"], inplace=True)

### Exporting Package Names

In [78]:
peo_usage[["rating","Peo_TV_Package"]].drop_duplicates(["Peo_TV_Package"]).to_csv("data/product_catalog/Peo_Names.csv")

In [79]:
rating_df=peo_usage[["ACCOUNT_NUM.hash","Peo_TV_Package","rating"]]
rating_df.rename(columns={'rating':'ratings','Peo_TV_Package':'package'},inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [80]:
rating_df.reset_index(inplace=True)
rating_df.drop("index", axis=1, inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [82]:
rating_df = rating_df.groupby(["ACCOUNT_NUM.hash","package"]).agg({'ratings':sum})
rating_df.reset_index(inplace=True)

In [86]:
rating_df.head()

,ACCOUNT_NUM.hash,package,ratings
0,0003102d10a11c8f2f0f241f00a36b2d,PEO_SILVER_FTTH,0.0
1,0003102d10a11c8f2f0f241f00a36b2d,PEO_UTHAYAM,1.8
2,00092b1fd47eb05d000a3553ae072056,PEO_SILVER,6.8
3,00094326efae1d73371d118204e12e46,PEO_SILVER,4.0
4,000f325a72b9d24742237070939b57d1,PEO_SILVER,6.4


In [87]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35975 entries, 0 to 35974
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ACCOUNT_NUM.hash  35975 non-null  object 
 1   package           35975 non-null  object 
 2   ratings           35975 non-null  float64
dtypes: float64(1), object(2)
memory usage: 843.3+ KB


In [88]:
rating_df.to_csv("data/azure/peoTV_user_ratings.csv")

## User content

In [14]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [15]:
user_profile = user_profile[(user_profile["Peo_TV_Package"] != "NO_INFO")]
user_profile.reset_index(drop=True, inplace=True)

In [16]:
user_profile.shape

(19095, 113)

## TODO : Feature selection #####

In [17]:
user_profile.drop(["Peo_TV_Package","PEO_TV_Categorical","PEO_TV_Scaled","BIRTH_YEAR","Megaline_Month_Subscribed","LTE_Month_Subscribed","VAS_Month_Subscribed"], axis=1, inplace= True)

In [18]:
user_profile.to_csv("data/azure/peoTV_user_content.csv")

## Product (package) content

In [19]:
product_profile = pd.read_csv("data/product_catalog/Product_Profile_Finalised_CSV.csv")

In [20]:
product_df=product_profile.iloc[:,[0,2,3,4,6,7,15,16,24,25,26,27,28,29,30,31,32,33,34,35,37,38,39,40]]
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Product_DB_ID                248 non-null    object 
 1   Base_Type                    253 non-null    object 
 2   Pricing_Type                 146 non-null    object 
 3   Package_Type                 146 non-null    object 
 4   Title                        145 non-null    object 
 5   Description                  59 non-null     object 
 6   Price                        145 non-null    object 
 7   Monthly_Rental               149 non-null    float64
 8   PEOTV_No_of_Channels         16 non-null     float64
 9   PEOTV_Foreign_Channels       16 non-null     float64
 10  PEOTV_Local_Channels         16 non-null     float64
 11  PEOTV_Movie_Channels         16 non-null     float64
 12  PEOTV_Kids_Channels          16 non-null     float64
 13  PEOTV_Religious_Chan

In [21]:
temp_package_df=rating_df["package"]

In [22]:
peo_tv_products_df = product_df.merge(temp_package_df.drop_duplicates(), left_on=['Product_DB_ID'], right_on=['package'] ,
                   how='left', indicator=True)

In [23]:
product_df = peo_tv_products_df[(peo_tv_products_df["_merge"]=="both")]

In [24]:
product_df.drop(["package","_merge"],axis=1,inplace=True)
product_df.rename(columns={'Product_ID':'package'},inplace=True)
product_df.reset_index(drop=True, inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [25]:
cols =["PEOTV_Foreign_Channels","PEOTV_Local_Channels","PEOTV_Movie_Channels","PEOTV_Kids_Channels","PEOTV_Religious_Channels","PEOTV_Tamil_Channels","PEOTV_Educational_Channels","PEOTV_Music_Channels","PEOTV_Sports_Channels","PEOTV_News_Channels"]

In [26]:
product_df[cols] = product_df[cols].fillna(0)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [27]:
product_df[["PEOTV_No_of_Channels","Monthly_Rental"]] = product_df[["PEOTV_No_of_Channels","Monthly_Rental"]].fillna(0)

In [28]:
scaler = MinMaxScaler()
for col in cols:
    product_df[col] = scaler.fit_transform(product_df[col].values.reshape(-1,1))

/var/folders/00/gmvdw06n48lgtsz126ng5y2m0000gn/T/ipykernel_23978/2295505821.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df[col] = scaler.fit_transform(product_df[col].values.reshape(-1,1))
/var/folders/00/gmvdw06n48lgtsz126ng5y2m0000gn/T/ipykernel_23978/2295505821.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df[col] = scaler.fit_transform(product_df[col].values.reshape(-1,1))
/var/folders/00/gmvdw06n48lgtsz126ng5y2m0000gn/T/ipykernel_23978/2295505821.py:3: SettingWithCopyWarni

In [29]:
product_df.rename(columns={"Product_DB_ID":"package"},inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [30]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   package                      34 non-null     object 
 1   Base_Type                    34 non-null     object 
 2   Pricing_Type                 34 non-null     object 
 3   Package_Type                 34 non-null     object 
 4   Title                        33 non-null     object 
 5   Description                  34 non-null     object 
 6   Price                        34 non-null     object 
 7   Monthly_Rental               34 non-null     float64
 8   PEOTV_No_of_Channels         34 non-null     float64
 9   PEOTV_Foreign_Channels       34 non-null     float64
 10  PEOTV_Local_Channels         34 non-null     float64
 11  PEOTV_Movie_Channels         34 non-null     float64
 12  PEOTV_Kids_Channels          34 non-null     float64
 13  PEOTV_Religious_Channe

In [32]:
product_df.to_csv("data/azure/peoTV_package_content.csv")